In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Weaviate
from langchain.document_loaders import TextLoader

from langchain.document_loaders import TextLoader

loader = TextLoader("./wonders.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [2]:
docs

[Document(page_content="Seven Wonders of the World\nThe Seven Wonders of the World are a group of places around the globe that are considered to be of great importance. These are: The Colosseum in Italy, Petra in Jordan, ChichÃ©n ItzÃ¡ in Mexico, Christ the Redeemer in Brazil, Machu Picchu in Peru, Taj Mahal in India and The Great Wall of China.\nDownload Arrow\nDownload FREE teacher-made resources covering 'Seven Wonders of the World'\nView FREE Resources Chevron Right\nWhat are the Seven Wonders of the World?\nThe original seven wonders, also referred to as the Seven Wonders of the Ancient World, were:\n\nThe Great Pyramid of Giza, Egypt;\nThe Hanging Gardens of Babylon;\nThe Statue of Zeus at Olympia;\nThe Temple of Artemis at Ephesus;\nThe Mausoleum at Halicarnassus;\nThe Colossus of Rhodes;\nThe Lighthouse of Alexandria.\nThe Seven Wonders of the Ancient World were remarkable, humanly-constructed landmarks from ancient, classical civilisations. This original list dates from the se

In [3]:
embeddings = OpenAIEmbeddings()

db = Weaviate.from_documents(docs, embeddings, weaviate_url=os.getenv("WEAVIATE_URL"), by_text=False)

In [7]:
query = "what is the first wonder?"
retriever = db.as_retriever(search_type="mmr")
retriever.get_relevant_documents(query)[0]

Document(page_content="Seven Wonders of the World\nThe Seven Wonders of the World are a group of places around the globe that are considered to be of great importance. These are: The Colosseum in Italy, Petra in Jordan, ChichÃ©n ItzÃ¡ in Mexico, Christ the Redeemer in Brazil, Machu Picchu in Peru, Taj Mahal in India and The Great Wall of China.\nDownload Arrow\nDownload FREE teacher-made resources covering 'Seven Wonders of the World'\nView FREE Resources Chevron Right\nWhat are the Seven Wonders of the World?\nThe original seven wonders, also referred to as the Seven Wonders of the Ancient World, were:\n\nThe Great Pyramid of Giza, Egypt;\nThe Hanging Gardens of Babylon;\nThe Statue of Zeus at Olympia;\nThe Temple of Artemis at Ephesus;\nThe Mausoleum at Halicarnassus;\nThe Colossus of Rhodes;\nThe Lighthouse of Alexandria.\nThe Seven Wonders of the Ancient World were remarkable, humanly-constructed landmarks from ancient, classical civilisations. This original list dates from the sec

In [8]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.llms import OpenAI

In [9]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    OpenAI(temperature=0), chain_type="stuff", retriever=db.as_retriever()
)

In [10]:
chain(
    {"question":  "what is the first wonder?"},
    return_only_outputs=True,
)

{'answer': ' The first wonder is The Great Pyramid of Giza in Egypt.\n',
 'sources': './wonders.txt'}

In [ ]:
import weaviate
import os

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
auth_client_secret = (weaviate.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY")),)
client = weaviate.Client(
    url=WEAVIATE_URL,
    additional_headers={
        "X-Openai-Api-Key": os.getenv("OPENAI_API_KEY"),
    },
)